## 为什么要用卷积神经网络
- 传统的网络需要大量的参数，但是这些参数是否重复了，例如，我们识别一个人，只要看到眼睛、鼻子和嘴就基本能识别出来，往往`局部特征就可以做出判断，并不需要全部特征`。
- 平移不变性
- 通过卷积的计算操作来提取图像局部特征，每一层都会计算一些局部特征，这写局部特征再汇总到下一层，一层一层的传递下去，特征由小变大，最后再通过这些局部的特征对图片进行处理，这样大大提供计算效率，提高准确度

## 卷积介绍
### 卷积计算
- 卷积核Kernel: 也称作为过滤器filter，一般是一个3*3或者5*5的权重矩阵W。
- 输入矩阵使用卷积核W进行滑动，每滑动一步，将所覆盖的值与矩阵对应的值相乘，求和做为输出矩阵的一项，依次类推。5*5的矩阵不做padding，输出则是3*3的新矩阵
- 边界填充Padding: 对输入矩阵上下左右各加padding层
- 步长Stride: 指定每次滑动几个距离
- 输出矩阵大小: $output\_size=1+\frac{input\_size+2*padding-kernel\_size}{stride}$

### 卷积层
- 每一个卷积层会设置多个核，每个核代表不同的特征
- 训练过程就是训练这写不同的核
- 卷积的操作是线性的，所以也需要激活函数，一般情况下使用relu
- [卷积层计算细节](https://zhuanlan.zhihu.com/p/29119239)

### 池化层pooling
- 减少卷积层之间的连接，降低运算复杂程度，减少空间大小
- 与卷积操作一样，卷积核覆盖的区域进行合并，只保留一个值
- 合并的方式: 
    - max_pooling:取最大值
    - avg_pooling:取平均值
- 输出矩阵大小: $output\_size=1+\frac{input\_size-kernel\_size}{stride}$

### dropout层
- 防止过拟合，增强模型的泛化能力
- 按照一定的概率随机将一部分神经网络单元暂时从网络中丢弃

### 全链接层
- 一般作为最后的输出层使用
- 将特征进行压扁，变成一维向量
- 分类用softmax，回归用linear即可

[官方案例](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,6,3) # 1个输入通道，6个输出通道，3*3的卷积核
        self.conv2=nn.Conv2d(6,16,3) # 6个输入通道，16个输出通道，3*3的卷积核
        self.fc1=nn.Linear(16*6*6,120)  #16*6*6是输入的特征数，120是输出的特征数 y=Wx+b
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2)) 
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=x.view(-1,self.num_flat_features(x)) # reshape
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
        
    def num_flat_features(self,x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        

In [8]:
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [9]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1.weight 

10
torch.Size([6, 1, 3, 3])


In [10]:
input=torch.randn(1,1,32,32) 
out=net(input)
print(out)

tensor([[ 0.1074,  0.0972, -0.0027, -0.0031, -0.0776, -0.1223, -0.0472, -0.0660,
          0.0020,  0.0695]], grad_fn=<AddmmBackward>)


In [11]:
net.zero_grad()
out.backward(torch.randn(1,10))  

In [12]:
# loss function
output=net(input)
target=torch.randn(10)
target=target.view(1,-1)

criterion=nn.MSELoss()
loss=criterion(output,target)
print(loss)

tensor(1.0321, grad_fn=<MseLossBackward>)


In [15]:
"""input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss"""
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [17]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0052,  0.0051, -0.0123,  0.0139,  0.0150, -0.0265])


In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update